In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/NL/"

In [3]:
files = sorted(os.listdir(folder))
len(files)

1

In [4]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

49

In [5]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/49 [00:00<?, ?it/s]

In [6]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(49, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,FEEST- en ié/ cnbe be J^one bes menfctjen faï ...,@@@@@@@@@@@FEEST- en ié/ cnbe be J^one bes men...,11 Want 8 FEEST- EN is/ ende de @Sone des men...
1,de Gebeurtenisfen in 1787 enz. voorgevallen. 1...,@@@@@de Gebeurtenisfen in 1787 enz. voorgevall...,I83 de Gebeurtenissen in I787 enz. voorgevall...
2,"94 D. VAN GESSCHER, OVER ’T GENEEZEN DER en de...","@94 D. VAN GESSCHER, OVER ’T GENEEZEN DER en d...","94 D. VAN GESSCHER, OVER 'T GENEEZEN DER en d..."
3,'gtftcrefi vooiby dcCe Stad nyt Holland kömtnd...,'gtftcrefi vooiby dcCe Stad nyt Holland kömtnd...,gistere@n voorby dese Stad uyt Holland komend...
4,LYDE NS-STOFFEN. 49 6 nu IMatuö ban $aïiïca lj...,@@@@@@@@@@@@LYDE NS-STOFFEN. @49 6 nu @IMatuö ...,D 14 Gy 49 LYDE@NS-STOFFEN. 6 Als nu Pilatus ...


In [7]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,49.000000,49.000000,49.000000
mean,4330.265306,4671.775510,4670.775510
std,3972.999773,4509.177444,4509.177444
min,522.000000,537.000000,536.000000
25%,1309.000000,1326.000000,1325.000000
50%,1915.000000,2031.000000,2030.000000
75%,6308.000000,7216.000000,7215.000000
max,12915.000000,17773.000000,17772.000000


In [8]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

157


In [9]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count      49.000000
mean      187.040154
std       752.935848
min         2.390767
25%         8.582817
50%        24.123422
75%        42.981607
max      3790.000000
Name: cer, dtype: float64

In [10]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    49.000000
mean     27.302321
std      24.080204
min       2.380952
25%       8.519953
50%      23.441227
75%      38.888889
max      97.946768
Name: cer, dtype: float64

In [11]:
data.to_pickle("../../data/nl/data/test.pkl")